In [1]:
import sys
import gpytorch
import numpy as np
import pandas as pd
import torch
from datetime import datetime
from scipy.stats import qmc
import itertools
from itertools import combinations_with_replacement, combinations, permutations

import bo_methods_lib
from bo_methods_lib.bo_methods_lib.GPBO_Classes_New import * #Fix this later
from bo_methods_lib.bo_methods_lib.GPBO_Class_fxns import * #Fix this later
from bo_methods_lib.bo_methods_lib.analyze_data import * #Fix this later
from bo_methods_lib.bo_methods_lib.GPBO_Classes_plotters import * #Fix this later
import pympler
import pickle
import signac

from pympler import asizeof

from matplotlib import pyplot as plt

import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 200

#Ignore inconcistent version warning
import warnings
# from sklearn.exceptions import InconsistentVersionWarning
# warnings.filterwarnings(action='ignore', category=InconsistentVersionWarning)

/afs/crc.nd.edu/user/m/mcarlozo/.conda/envs/Toy_Problem_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
#From signac
import signac
meth_name_val_list = [1, 2, 3, 4, 5, 6]

criteria_dict = {"cs_name_val" : 1,
                 "ep_enum_val": {"$gte": 1},
                 "meth_name_val": {"$in": meth_name_val_list}}

criteria_dict = {"cs_name_val" : 10,
                 "meth_name_val": {"$in": meth_name_val_list}}


In [7]:
project = signac.get_project()
save_csv = True
save_figs = False
analyzer = General_Analysis(criteria_dict, project, save_csv)
plotters = Plotters(analyzer, save_figs)
# df_all_jobs, job_list, theta_true = analyzer.get_df_all_jobs()
df_best, job_list_best = analyzer.get_best_data()

for i in range(len(job_list_best)): 
    print(job_list_best[i].id)
    
df_best

098775ca7c4adff1f451c798e7dad174
6499d05b55cf9990374579c43c7c7441
bda35ff3265d681fa6a7d1a765362cfa
6aca188e81fa4f2a3499fc065ef6702f
a95b6a1a303b0213c5a9c61977783762
c6e323381de4c2e3cd1a3dab1b1c0bc3


,Run Number,BO Iter,Best Error,Exploration Bias,Max EI,Theta Max EI,Min Obj,Min Obj Act,Theta Min Obj,Min Obj Cum.,...,Speed,Time/Iter,EP Method Val,MSE,BO Method,Job ID,Max Evals,Termination,Total Run Time,L2 Norm Theta
9,4,3,0.553657,1.0,1.807488e+00,"[-208.3519265544624, -98.23382713326507, -157....",-1.253831,0.061223,"[-208.35490546715042, -98.22792799256493, -157...",0.061223,...,0.009517,20.068084,CONSTANT,0.002449,A1,098775ca7c4adff1f451c798e7dad174,7,ei-regret,140.476590,15.306554
333,3,7,-1.827367,1.0,0.000000e+00,"[-122.58801586032308, -96.58584442718383, -216...",-1.945035,-1.996556,"[-179.15443638001562, -58.87071282403899, -158...",-1.996556,...,0.001192,16.901382,CONSTANT,0.005432,B1,6499d05b55cf9990374579c43c7c7441,7,ei-regret,118.309677,47.506052
578,3,28,0.013762,1.0,8.294301e-03,"[-208.9352690917109, -99.88550626416668, -180....",0.000615,0.000658,"[-199.50654069920282, -99.30752329332982, -169...",0.000658,...,0.000447,4307.279348,CONSTANT,0.000026,A2,bda35ff3265d681fa6a7d1a765362cfa,50,max_budget,215363.967395,0.851554
338,5,5,-2.877016,1.0,3.531015e+01,"[-243.27099775398395, -149.78613923931056, -16...",-4.727424,-5.536549,"[-194.4509987080909, -94.71492770151742, -167....",-5.536549,...,0.024513,17012.453856,CONSTANT,0.000158,B2,6aca188e81fa4f2a3499fc065ef6702f,30,obj,510373.615687,8.186842
489,3,34,0.009131,1.0,1.822197e-01,"[-212.7900168505276, -97.91727684609032, -174....",0.000060,0.000307,"[-199.2121585024204, -100.34932448766874, -168...",0.000307,...,0.000498,5817.783509,CONSTANT,0.000012,C2,a95b6a1a303b0213c5a9c61977783762,49,obj-regret,285071.391930,1.342044
644,3,13,0.021307,1.0,1.874823e-07,"[-195.69062320820856, -87.27659902332121, -156...",0.000465,0.002625,"[-194.59937408257997, -99.65405294045762, -169...",0.002625,...,0.000574,1099.435167,CONSTANT,0.000105,D2,c6e323381de4c2e3cd1a3dab1b1c0bc3,28,obj-regret,30784.184685,5.415666


In [ ]:
plotters.plot_one_obj_all_methods("min_sse")

In [ ]:
plotters.plot_objs_all_methods(["min_sse", "sse"])

In [ ]:
plotters.plot_hypers(job_list_best[0], title = None)

In [ ]:
plotters.plot_thetas(job_list_best[0], "min_sse", title = None)

In [ ]:
# title = "Heat Map Pair " + z_choice
levels = [100, 100, 100, 100, 100, 100]
pair = 0
z_choice = "sse_mean"
plotters.plot_hms_all_methods(pair, z_choice, levels, log_data = False, title = None)

In [ ]:
levels = [100, 100, 100, 100]
pair = 0
z_choice = ["sse_sim", "sse_mean", "sse_var", "ei"]
plotters.plot_hms_gp_compare(job_list_best[0], df_best["Run Number"].iloc[0], df_best["BO Iter"].iloc[0], 
                             pair, z_choice, levels, log_data = False, title = None)

In [ ]:
#Make Heat maps for all pairs
z_choices = ["sse_sim", "sse_mean", "sse_var", "ei"]
levels = [100,100,100,100]

#Loop over best jobs
for i in range(len(job_list_best)):   
    #Get jobs, runs, and iters to examine
    job = job_list_best[i]
    run_num = df_best["Run Number"].iloc[i]
    bo_iter = df_best["BO Iter"].iloc[i]
    
    #Back out number of parameters
    string_val = df_best["Theta Min Obj"].iloc[0]
    try:
        numbers = [float(num) for num in string_val.replace('[', '').replace(']', '').split()]
    except:
        numbers = [float(num) for num in string_val]
        
    #Create list of parameter pair combinations
    dim_theta = len(np.array(numbers).reshape(-1, 1))
    dim_list = np.linspace(0, dim_theta-1, dim_theta)
    pairs = len((list(combinations(dim_list, 2))))
    
    #Loop over parameter pairs
    for pair in range(pairs):
        plotters.plot_hms_gp_compare(job, run_num, bo_iter, pair, z_choices, levels)